In [1]:
import cv2
import numpy as np

# Load the image
image_path = './Shapes.jpg'
image = cv2.imread(image_path)
cv2.imshow("Original Image", image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
#blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 200)

# Find contours
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Function to detect shape name
def detect_shape(c):
    shape = "unidentified"
    peri = cv2.arcLength(c, True)
    # Adjust the epsilon value to improve accuracy
    epsilon = 0.02 * peri  # Experiment with this value
    approx = cv2.approxPolyDP(c, epsilon, True)
    
    if len(approx) == 3:
        shape = "triangle"
    elif len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        ar = w / float(h)
        shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
    elif len(approx) == 5:
        shape = "pentagon"
    elif len(approx) == 6:
        shape = "hexagon"
    else:
        shape = "circle"
    
    return shape


# Create copies for both tasks
labeled_image = image.copy()
quadrilateral_image = image.copy()

# Detect shapes and label them
for contour in contours:
    shape_name = detect_shape(contour)
    
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    cv2.drawContours(labeled_image, [contour], -1, (0, 255, 0), 2)
                                                    # contour colors
    cv2.putText(labeled_image, shape_name, (cX - 20, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                                                                                            # text color
# Remove non-quadrilateral shapes
for contour in contours:
    shape_name = detect_shape(contour)
    if shape_name not in ["square", "rectangle"]:
        cv2.drawContours(quadrilateral_image, [contour], -1, (0, 0, 0), -1)

# Save and display the images
cv2.imwrite('./labeled_shapes.jpg', labeled_image)
cv2.imwrite('./quadrilateral_shapes.jpg', quadrilateral_image)

cv2.imshow("Labeled Shapes", labeled_image)
cv2.imshow("Quadrilateral Shapes", quadrilateral_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


<p dir = "rtl">
    ابتدا شکل اصلی را خوانده و نمایش می دهیم. سپس در ادامه تصور را سیاه و سفید می کنیم و با اعمال یک فیلتر گاسین و ادج دیتکشن کنی میایم و کانتور های عکس را پیدا می کنیم.
    </p>
    <p dir = "rtl">
    یک تابع detect_shape داریم که با استفاده از تعداد خمیدگی تعداد ضلع ها را می شمارد و بر اساس تعداد ضلع نام های آن ها را در خروجی تابع قرار می دهد.
    در ادامه نیز نوشته های سفید رنگ که اسم های شکل هاست بر روی آنها اضافه می کند و عکس را نمایش می دهد
</p>
<p dir = "rtl">
    در آخر هم با توجه به اینکه شکل ها را تشخیص داده شکل های چهارضلعی مربع و مستطیل را نگه می دارد و داخل باقی شکل ها را سیاه می کند به منظور اینکه آن شکل ها را حذف کرده است.
    </p>
    
<p dir = "rtl">
    نکته مهم و تست شده در این کد این است که کد دارای باگی است که ما قادر به رفع آن نشدیم و این مشکل تشخیص یکی از مثلث ها به عنوان شش ضلعی است که با تست هایی که انجام دادیم دیدیم اگر فیلتر گاسین 3 در 3 اعمال کنیم یکی از مثلث ها حذف می شود و دیگری هم با فیلتر گاسین 5 در 5 مثلث دیگر حذف می شود.
    می توان از اشتراک این دو حالت چهارضلعی ها را به صورت ایده آل داشت اما تضمینی نیست که در عکس دیگری همین عملکرد را داشته باشد به همین خاطر راه ثابت و منطقی ای نیست.
    </p>

In [2]:
# Farms 1
import cv2
import numpy as np

# Load the image
image_path = './FARMS.png'
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
edges = cv2.Canny(blurred, 50, 150)

# Find contours
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Compute area of each contour
areas = [(cv2.contourArea(c), c) for c in contours]
areas.sort(key=lambda x: x[0], reverse=True)

# Create a copy of the image to draw on
labeled_image = image.copy()

# Assign ranks based on area
for rank, (area, contour) in enumerate(areas, 1):
    M = cv2.moments(contour)
    if M["m00"] == 0:
        continue
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    cv2.drawContours(labeled_image, [contour], -1, (0, 255, 0), 2)
    cv2.putText(labeled_image, str(rank), (cX - 10, cY + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Save and display the labeled image
cv2.imwrite('./labeled_farms.jpg', labeled_image)
cv2.imshow("Labeled Farms", labeled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
# Farms 2
import cv2
import numpy as np

# Load the image
image_path = './FARMS_2.jpeg'
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)

# Find contours
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Compute area of each contour
areas = [(cv2.contourArea(c), c) for c in contours]
areas.sort(key=lambda x: x[0], reverse=True)

# Create a copy of the image to draw on
labeled_image = image.copy()

# Assign ranks based on area
for rank, (area, contour) in enumerate(areas, 1):
    M = cv2.moments(contour)
    if M["m00"] == 0:
        continue
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    cv2.drawContours(labeled_image, [contour], -1, (0, 255, 0), 2)
    cv2.putText(labeled_image, str(rank), (cX - 10, cY + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# Save and display the labeled image
cv2.imwrite('./labeled_farms.jpg', labeled_image)
cv2.imshow("Labeled Farms", labeled_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


<p dir = "rtl">
    ابتدا شکل اصلی را خوانده. سپس در ادامه تصور را سیاه و سفید می کنیم و با اعمال یک فیلتر گاسین و ادج دیتکشن کنی میایم و کانتور های عکس را پیدا می کنیم.
    </p>
    <p dir = "rtl">
    سپس مساحت هر ناحیه از کانتور ها را پیدا کرده و بر اساس لاندا به صورت reverse یعنی از ناحیه بزرگ به ناحیه کوچک عدد گذاری می کند و با فونت آبی آن ها را داخلش می نویسد.  
</p>

<p dir = "rtl">
    نکته مهم و تست شده در این کد این است که کد دارای باگی است که ما قادر به رفع آن نشدیم و این مشکل تشخیص زمین ها است که کم و زیاد مشخص می کند و علت این زیاد مشخص کردن مرز های کلفت زمین ها است و از طرفی هم اگر فیلتر اعمالی را در کرنل های بزرگتر اعمال کنیم دقت لبه یابی پایین می آید و همین باعث شمارش کم و یا زیاد زمین ها می شود.
    </p>

<p dir="rtl">
    کتابخانه مورد نیاز برای دستور زیر را در anaconda prompt یا ترمینال لینوکس وارد کنید:
    </p>
    <p>
    pip install opencv-python-headless torch torchvision pillow tkinter
    </p>
    <p>
    conda install pytorch torchvision torchaudio cpuonly -c pytorch
    </p>
    <p>
    https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b
    </p>
    <p>
    برای نصب پای تورچ، پیلو برای خواندن عکس و تینکر برای محیط gui این بخش
    </p>

In [4]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import torch
from torchvision import models, transforms
import cv2

# Load the pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()

# Define image transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Read the class labels
class_labels = {}
with open('imagenet_classes.txt', 'r') as f:
    for line in f:
        index, label = line.strip().split(', ')
        class_labels[int(index)] = label

def select_image():
    # Function to select an image file and display it
    file_path = filedialog.askopenfilename()
    if file_path:
        image = Image.open(file_path)
        image = ImageTk.PhotoImage(image)
        panel.configure(image=image)
        panel.image = image
        root.filename = file_path

def capture_image():
    # Function to capture an image from the webcam and display it
    ret, frame = cap.read()
    cv2.imwrite('capture.jpg', frame)
    image = Image.open('capture.jpg')
    image = ImageTk.PhotoImage(image)
    panel.configure(image=image)
    panel.image = image
    root.filename = 'capture.jpg'

def classify_image():
    # Function to classify the displayed image
    image_path = root.filename
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    # Map the predicted index to the corresponding class name
    predicted_class = class_labels[predicted.item()]
    result.config(text=f'Predicted class: {predicted_class}')

# Set up the GUI
root = tk.Tk()
root.title("Image Classifier")

# Add GUI elements
panel = tk.Label(root)
panel.pack()
select_button = tk.Button(root, text='Select Image', command=select_image)
select_button.pack()
capture_button = tk.Button(root, text='Capture Image', command=capture_image)
capture_button.pack()
classify_button = tk.Button(root, text='Classify Image', command=classify_image)
classify_button.pack()
result = tk.Label(root, text='Classified class will appear here')
result.pack()

# Start the GUI
root.mainloop()

# Release the webcam
cap.release()


C:\Users\Asus\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Asus\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<p dir = "rtl">
    با استفاده از تینکر و پای تورچ و شبکه رس نت یک اپلیکیشن با زبان پایتون نوشته ایم که بر اساس فایل imagenet_classes.txt که از گیت هاب پیدا کردیم و اعداد در کنار آن نوشته است.
    تشخیص این برنامه و خروجی اصلی این تابع یک عدد است که با خواندن آن فایل مرتب شده بر اساس اعداد به جواب نام آن میرسیم و در خروجی نمایش می دهیم.
    عکس می تواند از روی کامپیوتر یا با استفاده از وب کم لپ تاپ به برنامه داده شود و برنامه با دکمه classify image بعد از اینکه عکس را از وب کم یا از کامپیوتر برداشت و به ما نشان داد نوع آن را تشخیص دهد.
    </p>